In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
import matplotlib.animation as animation

plt.style.use('seaborn-pastel')
%matplotlib notebook

In [ ]:
# константы

# i = 0+1j
# h_bar = 1.054571817e-34 # J/s
# m_e = 9.1093837015e-31 # kg

In [ ]:
def gauss(x, x_0, sigma, a=1):
    omega = 2
    k = 5
    def phase(x, x_0):
        return omega*(x-x_0)**3
    a = 5
#     w = np.cos(phase(x, a)) + 1j*np.cos(phase(x, a) + np.pi/2)
#     w = np.exp(1j*(k*x))
    w = np.cos(k*x) + 1j*np.sin(k*x)
    envelope = np.exp(-((x-x_0)/sigma)**2)
    out = w * envelope
    norm = sum(np.abs(out))
    return out/norm

def harmonic_oscillator(x, x_0, a):
    return a*(x-x_0)**2

def laplacian(psi):
    return filters.laplace(psi)

def get_d_psi(psi, V, dt):
    s1 = laplacian(psi)
    s2 = -i*V*psi/h_bar
    
    s2 = 0
    out = s1 + s2
    return out * dt

def normalize(psi):
    a = np.sum(np.abs(psi))
    return psi/a

# def run(frame, psi):
#     print('run')
#     dt = 1e3
#     d_psi = get_d_psi(psi, V, dt)
#     psi += d_psi
#     psi /= sum(np.abs(psi))
    
#     out = d_psi
    
#     line_abs.set_data(x, np.abs(out))
#     line_re.set_data(x, np.real(out))
#     line_im.set_data(x, np.imag(out))
    
#     ax.relim()
#     ax.autoscale_view(True,True,True)
#     ax.set_title('{}'.format(frame))
#     return line


In [ ]:
# задаем сетку
x_min = 0
x_max = 40
dx = 1e-2
x = np.arange(x_min, x_max, dx)


barier_height = 50
barier_width = 0.05
V = barier_height*((x > x_max/2) * (x < (x_max/2+barier_width)))
# V = harmonic_oscillator(x, (x_max+x_min)/2, 1e-10)
# V = np.zeros_like(x)


# начальная волновая функция

psi_0 = gauss(x, x_max/4, 2)
psi_r_0 = np.real(psi_0)
psi_i_0 = np.imag(psi_0)

psi_1 = psi_0
psi_r_1 = np.real(psi_1)
psi_i_1 = np.imag(psi_1)

# plt.plot(x, np.abs(psi_0))

# norm = sum(np.abs(psi_r+1j*psi_i))
# psi_i /= norm
# psi_r /= norm

# какие-то рандомные константы
h_bar = 1
m = 1
c1 = h_bar/(2*m)
c2 = 1/h_bar

# параметры расчета
dt = 0.99*dx**2
t_max = 2
t_N = int(t_max/dt)

print('x', len(x))
print('t_N', t_N)

In [ ]:
%%time

sol = []
norm = []

# чтоб не рисовать каждый шаг
steps_per_frame = 100

checkpoint = 0
steps = int(t_N/steps_per_frame)
for n in range(steps):
    current_progress = n/steps*100
    if current_progress - checkpoint >= 5:
        print('Progress: {:.1f}%'.format(current_progress))
        checkpoint = current_progress
    for _ in range(steps_per_frame):
        psi_i_f = psi_i_0 + c1*dt/(dx**2)*laplacian(psi_r_1) - c2*dt*V*psi_r_1
        psi_r_f = psi_r_0 - c1*dt/(dx**2)*laplacian(psi_i_1) + c2*dt*V*psi_i_1
        norm.append(sum(np.abs(psi_r_1+1j*psi_i_1)))
        
        nrm = sum(np.abs(psi_r_f+1j*psi_i_f))
        psi_i_f /= nrm
        psi_r_f /= nrm
        
        psi_i_0 = psi_i_1
        psi_i_1 = psi_i_f
        
        psi_r_0 = psi_r_1
        psi_r_1 = psi_r_f
        

    sol.append(psi_r_1+1j*psi_i_1)

# рисуем интеграл от волновой функции, чтоб заметить, если всё разошлось    
# plt.plot(norm)    
#         norm = sum(np.abs(psi_r+1j*psi_i))
#         psi_i /= norm
#         psi_r /= norm
#     sol.append(psi_r+1j*psi_i)

In [ ]:
# сохраняем в файл, на случай если крашнется

# filename = 'shrod_barier_1.txt'

# f = open(filename, 'wb')
# data = np.array(sol, dtype=np.complex64)
# np.save(f, sol)
# f.close()

In [ ]:
# читаем с файла

# sol = np.load(filename)

In [ ]:
%%time
# анимация


fig, ax = plt.subplots(figsize=(8,5))

ax.set_frame_on(False)
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)

a = 0.015
ax.set_ylim(-a/10, a)
line_abs, = ax.plot([],[], lw=1)
line_re, = ax.plot([],[], lw=1)
line_im, = ax.plot([],[], lw=1)
line_V, = ax.plot([],[], lw=1)
line_V.set_data(x, V/max(V)/2)


def run (frame, sol):
    ax.collections.clear()
    line_abs.set_data(x, np.abs(sol[frame]))
    ax.fill_between(x, np.abs(sol[frame]), facecolor='blue', alpha=0.03)
    line_re.set_data(x, np.real(sol[frame]))
    line_im.set_data(x, np.imag(sol[frame]))
    ax.relim()
    ax.autoscale_view(True,True,False)
    ax.set_title('{}'.format(frame))
    return line_abs

frames = len(sol)
ani = animation.FuncAnimation(fig, run, frames=frames, fargs=[sol], interval=10)
ani.save('schrod_new.mp4',fps=60, dpi=200)